---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Understanding and Predicting Property Maintenance Fines

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/).
___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant

___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric is the Area Under the ROC Curve (AUC); the target is a model with an AUROC of greater than 0.75.

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc

def read_input():
    ## read in all input files and create dfs for test and train
    
    ## the training set (all tickets issued 2004-2011)
    train = pd.read_csv(r'train.csv', usecols=('ticket_id', 'agency_name', 'inspector_name','violator_name',
                                                        'state', 'ticket_issued_date', 'hearing_date', 
                                                        'violation_code', 'violation_description', 'disposition',
                                                        'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 
                                                        'discount_amount', 'clean_up_cost', 'judgment_amount',
                                                        'payment_amount', 'balance_due', 'payment_date',
                                                        'payment_status', 'collection_status', 'grafitti_status',
                                                        'compliance_detail', 'compliance'), encoding = 'ISO-8859-1')

    ## the test set (all tickets issued 2012-2016)
    test = pd.read_csv(r'readonly/test.csv', usecols=('ticket_id', 'agency_name', 'inspector_name','violator_name',
                                                      'state', 'ticket_issued_date', 'hearing_date', 
                                                      'violation_code', 'violation_description', 'disposition',
                                                      'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 
                                                      'discount_amount', 'clean_up_cost', 'judgment_amount',
                                                      'grafitti_status'))
    ## mapping from ticket id to addresses
    address = pd.read_csv(r'readonly/addresses.csv')
    
    ## mapping from addresses to lat/lon coordinates. Note: misspelled addresses may be incorrectly geolocated
    latlon = pd.read_csv(r'readonly/latlons.csv')
        
    ## we don't need the null compliance items right now, but they might be useful later if we want to use
    ## a semi-supervised or unsupervised model
    nullcomply = train[train['compliance'].isnull()]
    train = train.dropna(subset=['compliance'])
    
    train = train.merge(address, on='ticket_id', how='inner').merge(latlon, on='address', how='inner')
    test = test.merge(address, on='ticket_id', how='inner').merge(latlon, on='address', how='inner')
    
    ## there are two citations in the training set that don't have lat/lon assigned (citations 89535 and 223598) and 
    ## five in test (317124, 329689, 329393, 333990, 367165).  just dropping them will make our result too short for 
    ## the autograder, so set all of them to the coordinates of detroit city hall.
    train = train.fillna({'lat':42.3295706, 'lon': -83.043828})
    test = test.fillna({'lat':42.3295706, 'lon': -83.043828})
    
    ## leaving this here because i had to look it up multiple times.  this will print selected columns based on a
    ## condition match.
    #print(test.loc[test['lat'].isnull(), ('ticket_id', 'address')])

    return train, test, nullcomply

def gaussian_nb(X_train, y_train, X_test, y_test, X_unknown):
    ## create classifier, fit, predict, generate roc auc for comparison
    nbclf = GaussianNB().fit(X_train, y_train)
   
    nb_pred = nbclf.predict_proba(X_test)
    
    fpr_nb, tpr_nb, _ = roc_curve(y_test, nb_pred[:,1].round(6))
    roc_auc_nb = auc(fpr_nb, tpr_nb)
    
    ## predictions on live test set
    nb_pred = nb_pred = nbclf.predict_proba(X_unknown)

    return roc_auc_nb, nb_pred

def random_forest(X_train, y_train, X_test, y_test, X_unknown):
    ## create classifier, fit, predict, generate roc auc for comparison
    rfclf = RandomForestClassifier().fit(X_train, y_train)
    
    rf_pred = rfclf.predict_proba(X_test)
    
    fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_pred[:,1].round(6))
    roc_auc_rf = auc(fpr_rf, tpr_rf)
    
    ## used to see which features are most weighted in the decision
    #print(rfclf.feature_importances_)

    ## predictions on live test set
    rf_pred = rfclf.predict_proba(X_unknown)

    return roc_auc_rf, rf_pred

def gradient_boosting(X_train, y_train, X_test, y_test, X_unknown):
    ## create classifier, fit, predict, generate roc auc for comparison
    gbclf = GradientBoostingClassifier().fit(X_train, y_train)
    
    gb_pred = gbclf.predict_proba(X_test)
    
    fpr_gb, tpr_gb, _ = roc_curve(y_test, gb_pred[:,1].round(6))
    roc_auc_gb = auc(fpr_gb, tpr_gb)
    
    ## used to see which features are most weighted in the decision
    #print(gbclf.feature_importances_)

    ## predictions on live test set
    gb_pred = gbclf.predict_proba(X_unknown)

    return roc_auc_gb, gb_pred

def blight_model():
    train, test, _ = read_input() ## change if we want to use nullcomply later
      
    ## paring down features that we care about.  first pass was just all numeric values to see what that would 
    ## get us before trying to deal with non-numeric values. subsequent lists were based on examining 
    ## feature_importances_ in the random forest classifier
    #floatdf = train[['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost',
     #                'judgment_amount', 'lat', 'lon']]
    #floatdf = train[['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost',
     #                'judgment_amount']]
    #floatdf = train[['fine_amount', 'late_fee', 'discount_amount','judgment_amount']]
    floatdf = train[['late_fee', 'discount_amount']]

    X_train, X_test, y_train, y_test = train_test_split(floatdf, train['compliance'].tolist())

    ## live test data, same feature set as above
    X_unknown = test[['late_fee', 'discount_amount']]

    ## nb returns ~0.70, rf ~0.76
    #nb_roc_auc, nb_pred = gaussian_nb(X_train, y_train, X_test, y_test, X_unknown)
    #print('gaussian nb auc = {}'.format(nb_roc_auc))
    
    rf_roc_auc, rf_pred = random_forest(X_train, y_train, X_test, y_test, X_unknown)
    #print('random forest auc = {}'.format(rf_roc_auc))
    
    ## haven't found a gb combination better than rf, and gb is considerably slower
    #gb_roc_auc, gb_pred = gradient_boosting(X_train, y_train, X_test, y_test, X_unknown)
    #print('gradient boosting auc = {}'.format(gb_roc_auc))
    
    ## if we find a better classifier than rf, update it here to generate compliance properly
    best_pred = rf_pred
    
    ## for a binary classifier, predict_proba returns a [k, 2] array.  a[0] is the probability of class 0,
    ## a[1] is the probability of class 1.  the two sum to 1.
    compliance = pd.Series(best_pred[:,1].round(6), test['ticket_id'])
    
    return compliance

In [45]:
#pd.options.display.max_rows = 250
#pd.options.display.max_columns = 250
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


random forest auc = 0.7516625814302372
gradient boosting auc = 0.7514878202684193


ticket_id
284932    0.061604
285362    0.025019
285361    0.070442
285338    0.061604
285346    0.070442
285345    0.061604
285347    0.059116
285342    0.318022
285530    0.025019
284989    0.029874
285344    0.059116
285343    0.025019
285340    0.025019
285341    0.059116
289828    0.029874
289830    0.036571
289829    0.036571
292133    0.025019
292134    0.059116
285349    0.070442
285348    0.061604
284991    0.029874
285532    0.029874
286073    0.029874
285406    0.029874
285001    0.029874
285006    0.025019
365862    0.966569
285405    0.025019
287857    0.017396
            ...   
376276    0.029874
376218    0.036571
376368    0.036571
376369    0.061604
376225    0.036571
376222    0.029874
376362    0.036571
376363    0.061604
376228    0.036571
376265    0.036571
376286    0.318022
376320    0.036571
376314    0.036571
376327    0.318022
376435    0.318022
376434    0.059116
376459    0.070442
376478    0.000000
376473    0.036571
376484    0.015856
376482    0.029874
37